In [61]:
import os
import copy
import numpy as np
import pandas as pd
import random

In [2]:
files = os.listdir('./Fourrier/')
print(files)
files2 = []
for i in range(int(len(files)/10)):
    files2.append(files[i*10])
print(files2)
Viable_Cols = ['Acceleration x (m/s^2)_kalman', 'Acceleration y (m/s^2)_kalman', 'Acceleration z (m/s^2)_kalman', 'Gyroscope x (rad/s)_kalman',
              'Gyroscope y (rad/s)_kalman', 'Gyroscope z (rad/s)_kalman', 'Linear Acceleration x (m/s^2)_kalman',
              'Linear Acceleration y (m/s^2)_kalman', 'Linear Acceleration z (m/s^2)_kalman']
Viable_Cols2 = []
Viable_Cols3 = copy.deepcopy(Viable_Cols)
for column in Viable_Cols:
    newCol = str(column+'_max_freq')
    Viable_Cols2.append(newCol)
    Viable_Cols3.append(newCol)

['Fourrier Austin Biking 30rpm.csv', 'Fourrier Austin Biking 30rpm.csv 0.png', 'Fourrier Austin Biking 30rpm.csv 1.png', 'Fourrier Austin Biking 30rpm.csv 2.png', 'Fourrier Austin Biking 30rpm.csv 3.png', 'Fourrier Austin Biking 30rpm.csv 4.png', 'Fourrier Austin Biking 30rpm.csv 5.png', 'Fourrier Austin Biking 30rpm.csv 6.png', 'Fourrier Austin Biking 30rpm.csv 7.png', 'Fourrier Austin Biking 30rpm.csv 8.png', 'Fourrier Austin Biking 60rpm.csv', 'Fourrier Austin Biking 60rpm.csv 0.png', 'Fourrier Austin Biking 60rpm.csv 1.png', 'Fourrier Austin Biking 60rpm.csv 2.png', 'Fourrier Austin Biking 60rpm.csv 3.png', 'Fourrier Austin Biking 60rpm.csv 4.png', 'Fourrier Austin Biking 60rpm.csv 5.png', 'Fourrier Austin Biking 60rpm.csv 6.png', 'Fourrier Austin Biking 60rpm.csv 7.png', 'Fourrier Austin Biking 60rpm.csv 8.png', 'Fourrier Austin Biking 90rpm.csv', 'Fourrier Austin Biking 90rpm.csv 0.png', 'Fourrier Austin Biking 90rpm.csv 1.png', 'Fourrier Austin Biking 90rpm.csv 2.png', 'Fourrier

In [57]:
window = 200
slide = 0.1
train_test = 0.8
def make_windows(window, slide, frame, train_test):
    jump = window*slide
    train_samples = []
    sparse_train = []
    test_samples = []
    sparse_test = []
    for i in range(int(len(frame['Acceleration x (m/s^2)_kalman'])/(window*slide))-int((1/slide))):
        #print(i*20+200)
        sample = frame.loc[(i*20):(i*20)+199, Viable_Cols]
        sample2 = []
        freqs = frame.loc[(i*20)+199, Viable_Cols2]
        sample2.append(freqs.tolist())
        mins = [np.min(sample[col]) for col in Viable_Cols]
        maxes = [np.max(sample[col]) for col in Viable_Cols]
        stdev = [np.std(sample[col], axis=0) for col in Viable_Cols]
        sample2.append(mins)
        sample2.append(maxes)
        sample2.append(stdev)
        if i*20 <= len(frame['Acceleration x (m/s^2)_kalman'])*train_test:
            train_samples.append(sample)
            sparse_train.append(sample2)
        else:
            test_samples.append(sample)
            sparse_test.append(sample2)
        
        test_samples = test_samples[9:]
        sparse_test = sparse_test[9:]
    return train_samples, sparse_train, test_samples, sparse_test
frame = pd.read_csv('./Fourrier/Fourrier Austin Biking 60rpm.csv')
print(int(len(frame['Acceleration x (m/s^2)_kalman'])/(window*slide))-10)
print(len(frame['Acceleration x (m/s^2)']))
print(window*slide)
#train_samples, sparse_train, test_samples, sparse_test = make_windows(window, slide, frame, train_test)

279
5786
20.0


In [123]:
window = 200
slide = 0.1
train_test = 0.8
def make_windows_PCA(window, slide, frame, train_test):
    Viable_Cols = ['0', '1', '2', '3', '4', '5']
    Viable_Cols2 = ['0_max_freq', '1_max_freq', '2_max_freq', '3_max_freq', '4_max_freq', '5_max_freq', ]
    jump = window*slide
    train_samples = []
    sparse_train = []
    test_samples = []
    sparse_test = []
    labels_train = []
    labels_test = []
    for i in range(int(len(frame['0'])/(window*slide))-int((1/slide))):
        sample = frame.loc[(i*jump):(i*jump)+(window-1), Viable_Cols]
        sample2 = []
        freqs = frame.loc[(i*jump)+199, Viable_Cols2]
        sample2.append(freqs.tolist())
        mins = [np.min(sample[col]) for col in Viable_Cols]
        maxes = [np.max(sample[col]) for col in Viable_Cols]
        stdev = [np.std(sample[col], axis=0) for col in Viable_Cols]
        sample2.append(mins)
        sample2.append(maxes)
        sample2.append(stdev)
        if 'SPM/RPM' in frame.columns:
            label = 'SPM/RPM'
        if 'RPM/SPM' in frame.columns:
            label = 'RPM/SPM'
        if 'RPM' in frame.columns:
            label = 'RPM'
        if 'SPM' in frame.columns:
            label = 'SPM'
        yval = frame[label][(i*jump)+199]
        if i*jump <= (len(frame['0'])*train_test):
            train_samples.append(sample)
            sparse_train.append(sample2)
            labels_train.append(yval)
        else:
            test_samples.append(sample)
            sparse_test.append(sample2)
            labels_test.append(yval)
    labels_test = labels_test[9:]
    test_samples = test_samples[9:]
    sparse_test = sparse_test[9:]
    return train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test

In [ ]:
files = os.listdir('./PCA Fourier/')
bike_train_samples, bike_sparse_train, bike_test_samples, bike_sparse_test, bike_labels_train, bike_labels_test = [],[],[],[],[],[]
walk_train_samples, walk_sparse_train, walk_test_samples, walk_sparse_test, walk_labels_train, walk_labels_test = [],[],[],[],[],[]
run_train_samples, run_sparse_train, run_test_samples, run_sparse_test, run_labels_train, run_labels_test = [],[],[],[],[],[]

Biking_Files = [0,1,2,16,17]
for i in Biking_Files:
    print('A')
    frame = pd.read_csv('./PCA Fourier/'+str(files[i]))
    train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_PCA(window, slide, frame, train_test)
    bike_train_samples.extend(train_samples)
    bike_sparse_train.extend(sparse_train)
    bike_test_samples.extend(test_samples)
    bike_sparse_test.extend(sparse_test)
    bike_labels_train.extend(labels_train)
    bike_labels_test.extend(labels_test)
Walking_Files = [10,11,12,13,14,18,20,21]
for i in Walking_Files:
    print('B')
    frame = pd.read_csv('./PCA Fourier/'+str(files[i]))
    train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_PCA(window, slide, frame, train_test)
    walk_train_samples.extend(train_samples)
    walk_sparse_train.extend(sparse_train)
    walk_test_samples.extend(test_samples)
    walk_sparse_test.extend(sparse_test)
    walk_labels_train.extend(labels_train)
    walk_labels_test.extend(labels_test)
Running_Files = [4,5,6,7]
for i in Running_Files:
    print('C')
    frame = pd.read_csv('./PCA Fourier/'+str(files[i]))
    train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_PCA(window, slide, frame, train_test)
    run_train_samples.extend(train_samples)
    run_sparse_train.extend(sparse_train)
    run_test_samples.extend(test_samples)
    run_sparse_test.extend(sparse_test)
    run_labels_train.extend(labels_train)
    run_labels_test.extend(labels_test)


    

In [ ]:
print(len(run_train_samples))
print(len(walk_train_samples))
print(len(bike_train_samples))
print(len(run_test_samples))
print(len(walk_test_samples))
print(len(bike_test_samples))
print(len(bike_labels_test))
print(run_train_samples[1])
print(run_sparse_train[1])


In [ ]:
def make_windows_full_dataset(window, slide, frame, train_test):
    Viable_Cols = ['0', '1', '2', '3', '4', '5']
    Viable_Cols2 = ['0_max_freq', '1_max_freq', '2_max_freq', '3_max_freq', '4_max_freq', '5_max_freq', ]
    jump = window*slide
    train_samples = []
    sparse_train = []
    test_samples = []
    sparse_test = []
    labels_train = []
    labels_test = []
    for i in range(int(len(frame['0'])/(window*slide))-int((1/slide))):
        sample = frame.loc[(i*jump):(i*jump)+(window-1), Viable_Cols]
        sample2 = []
        freqs = frame.loc[(i*jump)+199, Viable_Cols2]
        sample2.append(freqs.tolist())
        mins = [np.min(sample[col]) for col in Viable_Cols]
        maxes = [np.max(sample[col]) for col in Viable_Cols]
        stdev = [np.std(sample[col], axis=0) for col in Viable_Cols]
        sample2.append(mins)
        sample2.append(maxes)
        sample2.append(stdev)
        if frame['Still'][(i*jump)+199] == 1:
            yval = 1
        if frame['Walk'][(i*jump)+199] == 1:
            yval = 2
        if frame['Run'][(i*jump)+199] == 1:
            yval = 3
        if frame['Bike'][(i*jump)+199] == 1:
            yval = 4
        
        if i*jump <= (len(frame['0'])*train_test):
            train_samples.append(sample)
            sparse_train.append(sample2)
            labels_train.append(yval)
        else:
            test_samples.append(sample)
            sparse_test.append(sample2)
            labels_test.append(yval)
    labels_test = labels_test[9:]
    test_samples = test_samples[9:]
    sparse_test = sparse_test[9:]
    return train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test